# DH / DSPI

In [1]:
import csv
import re
#from io import StringIO
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics




In [2]:
#Funciones
#Separa columna tomada de logs en campos
def splitdata(col):
    temp = []
    for i in col:
        t = (i.split('='))
        temp.append(t[0])           
    return temp

#Normmaliza Dataset:
def splitdf(df):
    df = df.ix[:, ~df.columns.str.contains('Unnamed')]
    #r = re.compile(r"[^a-zA-Z0-9-]")
    df2 = pd.DataFrame(columns=df.columns)
    for i in df:
        new = df[i].str.split('=',n=1, expand=True)
        df2[i] = new[1]#.str.extract(r'([a-zA-Z0-9=])')
        #df2[i] = r.sub("",new[1])
    return df2

def fortidf(df):
    lcol = [] 
    lcol = df.columns
    df.columns = splitdata(lcol)
    df2 = splitdf(df)
    df2['ioc'] = 0
    return df2

In [57]:
df2 = pd.read_csv('Traffic.csv')
df = pd.read_csv('botnet.csv')
df = fortidf(df)
df2 = fortidf(df2)

#[^=]*

In [58]:
scolumns=['date', 'time', 'devid', 'vd', 'type', 'subtype', 'action',
       'app', 'appcat','craction', 'crscore', 'devname',
       'dstcountry', 'dstintf', 'dstip', 'dstport', 'duration', 'dvid','level','logid', 'logver', 'policyid','proto', 'rcvdbyte',
       'rcvdpkt', 'sentbyte', 'sentpkt', 'service', 'sessionid', 'srccountry',
       'srcintf', 'srcip', 'srcport', 'threatcnts', 'threatlvls', 'threats',
       'threattyps', 'threatwgts', 'trandisp', 'tranip', 'tranport', 'ioc']

In [59]:
df.loc[:,scolumns].head()

,date,time,devid,vd,type,subtype,action,app,appcat,craction,...,srcport,threatcnts,threatlvls,threats,threattyps,threatwgts,trandisp,tranip,tranport,ioc
0,2018-12-02,12:37:03,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,Botnet,1048576,...,1698,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,NaN,NaN,0
1,2018-12-02,12:36:56,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,Botnet,1048576,...,1695,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,NaN,NaN,0
2,2018-12-02,12:36:53,FGT80C3913606343,root,traffic,forward,deny,Conficker.Botnet,Botnet,1048576,...,1696,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,NaN,NaN,0
3,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,Botnet,1048576,...,1694,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,NaN,NaN,0
4,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,Botnet,1048576,...,1692,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,NaN,NaN,0


In [66]:
df2.loc[:,scolumns].head()

,date,time,devid,vd,type,subtype,action,app,appcat,craction,...,srcport,threatcnts,threatlvls,threats,threattyps,threatwgts,trandisp,tranip,tranport,ioc
0,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,None,...,58941,None,None,None,None,None,dnat,192.168.14.13,443,0
1,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,None,...,46764,None,None,None,None,None,dnat,192.168.14.13,443,0
2,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,None,...,50477,None,None,None,None,None,dnat,192.168.14.13,443,0
3,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,None,...,56050,None,None,None,None,None,dnat,192.168.14.13,443,0
4,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTP,unscanned,None,...,2744,None,None,None,None,None,dnat,192.168.14.13,80,0


In [63]:
data[['date', 'time','srcip','dstip','dstport','duration', 'rcvdbyte',
       'rcvdpkt', 'sentbyte', 'sentpkt', 'service','ioc' ]].sample(5)

,date,time,srcip,dstip,dstport,duration,rcvdbyte,rcvdpkt,sentbyte,sentpkt,service,ioc
73,2018-12-09,12:21:58,10.120.88.64,200.152.162.189,443,0,0,None,0,None,HTTPS,0
1,2018-12-09,12:21:59,45.237.106.242,200.41.197.1,443,14,112,2,244,5,HTTPS,0
15,2018-12-09,12:21:59,90.173.57.118,200.41.197.1,443,330,88,2,132,3,HTTPS,0
45,2018-12-09,12:21:58,10.63.21.190,40.121.16.193,443,11,7691,16,1208,13,HTTPS,0
50,2018-12-09,12:21:59,172.16.8.230,40.102.34.226,443,100,168,4,288,7,HTTPS,0


In [86]:
data = pd.concat([df,df2])
#data.shape
data.ioc

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
     ..
69    0
70    0
71    0
72    0
73    0
74    0
75    0
76    0
77    0
78    0
79    0
80    0
81    0
82    0
83    0
84    0
85    0
86    0
87    0
88    0
89    0
90    0
91    0
92    0
93    0
94    0
95    0
96    0
97    0
98    0
Name: ioc, dtype: int64

In [82]:
#Clasifica infectados:
data.loc[data.appcat == 'Botnet', 'ioc'] = 1
data.columns

Index(['action', 'app', 'appact', 'appcat', 'appid', 'applist', 'apprisk',
       'apps', 'bid', 'countapp', 'countweb', 'craction', 'crlevel', 'crscore',
       'date', 'devid', 'devname', 'dstcountry', 'dstintf', 'dstip', 'dstport',
       'duration', 'dvid', 'hostname', 'ioc', 'itime', 'level', 'logid',
       'logver', 'policyid', 'poluuid', 'proto', 'rcvdbyte', 'rcvdpkt',
       'saasinfo', 'sentbyte', 'sentpkt', 'service', 'sessionid', 'srccountry',
       'srcintf', 'srcip', 'srcport', 'subtype', 'threatcnts', 'threatlvls',
       'threats', 'threattyps', 'threatwgts', 'time', 'trandisp', 'tranip',
       'tranport', 'transip', 'transport', 'type', 'utmaction', 'utmref',
       'vd'],
      dtype='object')

In [84]:
X = data
#X.drop(['ioc'], inplace=True)
X.shape

(113, 59)

In [85]:
X.columns

Index(['action', 'app', 'appact', 'appcat', 'appid', 'applist', 'apprisk',
       'apps', 'bid', 'countapp', 'countweb', 'craction', 'crlevel', 'crscore',
       'date', 'devid', 'devname', 'dstcountry', 'dstintf', 'dstip', 'dstport',
       'duration', 'dvid', 'hostname', 'ioc', 'itime', 'level', 'logid',
       'logver', 'policyid', 'poluuid', 'proto', 'rcvdbyte', 'rcvdpkt',
       'saasinfo', 'sentbyte', 'sentpkt', 'service', 'sessionid', 'srccountry',
       'srcintf', 'srcip', 'srcport', 'subtype', 'threatcnts', 'threatlvls',
       'threats', 'threattyps', 'threatwgts', 'time', 'trandisp', 'tranip',
       'tranport', 'transip', 'transport', 'type', 'utmaction', 'utmref',
       'vd'],
      dtype='object')

In [98]:
train, test = train_test_split(data, test_size = 0.3,random_state=1234)

In [99]:
print(train.shape)
print(test.shape)

(79, 59)
(34, 59)


In [93]:
train_X = train.iloc[:, 1:31

AttributeError: 'DataFrame' object has no attribute 'diagnosis'

In [101]:
train

,action,app,appact,appcat,appid,applist,apprisk,apps,bid,countapp,...,time,trandisp,tranip,tranport,transip,transport,type,utmaction,utmref,vd
6,close,HTTPS,NaN,unscanned,NaN,NaN,NaN,NaN,44007393,NaN,...,12:21:59,dnat,192.168.14.13,443,NaN,NaN,traffic,NaN,NaN,REF131396
90,close,SMTP,NaN,Email,NaN,NaN,NaN,NaN,44007363,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1013387
77,close,Dropbox,NaN,Storage.Backup,NaN,NaN,NaN,NaN,44007367,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1004844
11,close,HTTPS,NaN,unscanned,NaN,NaN,NaN,NaN,44007398,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF131396
40,accept,DNS,NaN,Network.Service,NaN,NaN,NaN,NaN,44007393,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1008343
4,close,Conficker.Botnet,drop-session,Botnet,17201,block-p2p,critical,{Conficker.Botnet},100106761,1,...,12:36:49,snat,NaN,NaN,200.41.178.10,62108,traffic,block,BAQAAAAIAAAB3EAGuVQFY4B8L_ANcchABrFUBUOAfCvwDXA==,root
58,start,HTTPS,NaN,unscanned,NaN,NaN,NaN,NaN,44007371,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1006035
10,close,Conficker.Botnet,drop-session,Botnet,17201,block-p2p,critical,{Conficker.Botnet},100106702,1,...,12:36:26,snat,NaN,NaN,200.41.178.10,62099,traffic,block,BAQAAAAIAAAB3EAHqVAGg3x_0-wNcchAB6FQBoN8f9PsDXA==,root
37,close,Gmail,NaN,Email,NaN,NaN,NaN,NaN,44007401,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1008343
92,close,Google.Services,NaN,General.Interest,NaN,NaN,NaN,NaN,44007367,NaN,...,12:21:59,snat,None,None,NaN,NaN,traffic,NaN,NaN,REF1013387


In [102]:
model= DecisionTreeClassifier(random_state=1234)
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print("Accuracy:",metrics.accuracy_score(prediction,test_y))



NameError: name 'train_y' is not defined

In [44]:
df3 = df3.ix[:, ~df3.columns.str.contains('Unnamed')]

In [45]:
fortidf(df3)

,itime,date,time,devid,vd,type,subtype,action,app,appcat,...,srcport,threatcnts,threatlvls,threats,threattyps,threatwgts,trandisp,tranip,tranport,ioc
0,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,58941,None,None,None,None,None,dnat,192.168.14.13,443,0
1,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,46764,None,None,None,None,None,dnat,192.168.14.13,443,0
2,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,50477,None,None,None,None,None,dnat,192.168.14.13,443,0
3,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,56050,None,None,None,None,None,dnat,192.168.14.13,443,0
4,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTP,unscanned,...,2744,None,None,None,None,None,dnat,192.168.14.13,80,0
5,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,60339,None,None,None,None,None,snat,None,None,0
6,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,51606,None,None,None,None,None,dnat,192.168.14.13,443,0
7,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,51607,None,None,None,None,None,dnat,192.168.14.13,443,0
8,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,55106,None,None,None,None,None,dnat,192.168.14.13,443,0
9,1544368919,2018-12-09,12:21:59,FG-5KD3915800565,REF131396,traffic,forward,close,HTTPS,unscanned,...,64333,None,None,None,None,None,snat,None,None,0
